In [1]:
!pip install gym-super-mario-bros


[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [26]:
!pip install -U gym==0.21.0

  Using cached gym-0.21.0-py3-none-any.whl
  Attempting uninstall: gym
    Found existing installation: gym 0.26.2
    Uninstalling gym-0.26.2:
      Successfully uninstalled gym-0.26.2



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import math, random

import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd 
import torch.nn.functional as F

import matplotlib.pyplot as plt

import gym
import numpy as np

from collections import deque
from tqdm import trange

# Select GPU or CPU as device

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [3]:
gym.__version__

'0.21.0'

### Test it gym_super_mario_bros environment
It requires to use gym environment version : 0.21.0

In [10]:
from nes_py.wrappers import JoypadSpace
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)

done = True
for step in range(1000):
    if done:
        state = env.reset()
    state, reward, done, info = env.step(env.action_space.sample())
    env.render()

env.close()
print('Done')

### Create gym_super_mario_bros environment, prepare DQN for training

Next we set up a gym environment for the cartpole simulation, create a DQN model with Adam optimization, and create a replay buffer of length 1000.

In [24]:
# import numpy as np
# import random
# import torch
# import torch.nn as nn


# class CNNDQN(nn.Module):
#     def __init__(self, input_shape, num_actions):
#         super(CNNDQN, self).__init__()
#         self._input_shape = input_shape
#         self._num_actions = num_actions

#         self.features = nn.Sequential(
#             nn.Conv2d(input_shape[0], 32, kernel_size=8, stride=4),
#             nn.ReLU(),
#             nn.Conv2d(32, 64, kernel_size=4, stride=2),
#             nn.ReLU(),
#             nn.Conv2d(64, 64, kernel_size=3, stride=1),
#             nn.ReLU())

#         self.fc = nn.Sequential(
#             nn.Linear(self.feature_size, 512),
#             nn.ReLU(),
#             nn.Linear(512, num_actions))

#     def forward(self, x):
#         x = self.features(x).view(x.size()[0], -1)
#         return self.fc(x)

#     @property
#     def feature_size(self):
#         x = self.features(torch.zeros(1, *self._input_shape))
#         return x.view(1, -1).size(1)

#     def act(self, state, epsilon, device):
#         if random.random() > epsilon:
#             state = torch.FloatTensor(np.float32(state)).unsqueeze(0).to(device)
#             q_value = self.forward(state)
#             action = q_value.max(1)[1].item()
#         else:
#             action = random.randrange(self._num_actions)
#         return action